# Stock Predictor Work

## Import Packages

Importing necessary packages

In [11]:
import yfinance as yf
import datetime as dt
import pandas as pd

## Retreive Data from Yahoo Finance

In [31]:
# get daytime and previous 7 days

currentDT = dt.datetime.now()
pastDT = currentDT - dt.timedelta(days=7)
currentDT = str(currentDT)[:10]
pastDT = str(pastDT)[:10]

# retreive information for past 7 days, with 1 minute accuracy, for shell

company = "rds-b"
interval = "5m"

company = yf.download(company, start=pastDT, end=currentDT, interval=interval)
euro = yf.download("EURUSD=X", start=pastDT, end=currentDT, interval=interval)
oil = yf.download("EURUSD=X", start=pastDT, end=currentDT, interval=interval)

# build dataframe

data = pd.DataFrame()

data = data.assign(company_open = company['Open'])
data = data.assign(company_close = company['Close'])
data = data.assign(euro_open = euro['Open'])
data = data.assign(euro_close = euro['Close'])
data = data.assign(oil_open = oil['Open'])
data = data.assign(oil_close = oil['Close'])

data = data.assign(company_return = (company["Close"] - company["Open"]))
data = data.assign(euro_return = (euro["Close"] - euro["Open"]))
data = data.assign(oil_return = (euro["Close"] - euro["Open"]))

data

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


company_open  company_close  euro_open  euro_close  \
Datetime                                                                        
2020-05-26 09:30:00-04:00     31.170000      31.110001   1.097454    1.097574   
2020-05-26 09:35:00-04:00     31.110001      31.040001   1.097936    1.097213   
2020-05-26 09:40:00-04:00     31.030001      31.160000   1.097213    1.097815   
2020-05-26 09:45:00-04:00     31.155300      31.049999   1.098056    1.098177   
2020-05-26 09:50:00-04:00     31.040001      31.020000   1.097936    1.097936   
...                                 ...            ...        ...         ...   
2020-05-29 15:35:00-04:00     30.370001      30.389000   1.109509    1.109632   
2020-05-29 15:40:00-04:00     30.379999      30.370001   1.109632    1.109632   
2020-05-29 15:45:00-04:00     30.370001      30.270000   1.109878    1.109385   
2020-05-29 15:50:00-04:00     30.299999      30.370001   1.109509    1.109878   
2020-05-29 15:55:00-04:00     30.368799      30.450001   1.109878    1.110248   

                           oil_open  oil_close  company_return  euro_return  \
Datetime                                                                      
2020-05-26 09:30:00-04:00  1.097454   1.097574       -0.059999     0.000120   
2020-05-26 09:35:00-04:00  1.097936   1.097213       -0.070000    -0.000723   
2020-05-26 09:40:00-04:00  1.097213   1.097815        0.129999     0.000602   
2020-05-26 09:45:00-04:00  1.098056   1.098177       -0.105301     0.000121   
2020-05-26 09:50:00-04:00  1.097936   1.097936       -0.020000     0.000000   
...                             ...        ...             ...          ...   
2020-05-29 15:35:00-04:00  1.109509   1.109632        0.018999     0.000123   
2020-05-29 15:40:00-04:00  1.109632   1.109632       -0.009998     0.000000   
2020-05-29 15:45:00-04:00  1.109878   1.109385       -0.100000    -0.000493   
2020-05-29 15:50:00-04:00  1.109509   1.109878        0.070002     0.000369   
2020-05-29 15:55:00-04:00  1.109878   1.110248        0.081202     0.000370   

                           oil_return  
Datetime                               
2020-05-26 09:30:00-04:00    0.000120  
2020-05-26 09:35:00-04:00   -0.000723  
2020-05-26 09:40:00-04:00    0.000602  
2020-05-26 09:45:00-04:00    0.000121  
2020-05-26 09:50:00-04:00    0.000000  
...                               ...  
2020-05-29 15:35:00-04:00    0.000123  
2020-05-29 15:40:00-04:00    0.000000  
2020-05-29 15:45:00-04:00   -0.000493  
2020-05-29 15:50:00-04:00    0.000369  
2020-05-29 15:55:00-04:00    0.000370  

[312 rows x 9 columns]

So we have the values that we need. However, we might not be interested in the plain values, but in interactions between them. First, the label that is more interesting so far would be the return of the company at a specific time t. The return is calculated as follows. 

\begin{equation*}
Return(t) = [Close(t) - Open(t)] / EuroValue(t)
\end{equation*}

The return should be normalized, as the other inputs. 

For oil and euro_usd, we can estimate the trend of the value in the last n days, and then normalize it